In [23]:
import pandas as pd
import numpy as np


In [38]:
pip install SpellChecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for SpellChecker: filename=spellchecker-0.4-py3-none-any.whl size=3966499 sha256=92fc1777771f70179d2a5efa9a2fbde2e351211e72e56ccc6764c48c46af74b1
  Stored in directory: /root/.cache/pip/wheels/1c/cb/0f/5e7178a807a0648885bca7c40bdf2db39af2d9fc1adc9855c9
  Created wheel for inexactsearch: filename=inexactsearch-1.0.2-py3-none-any.whl size=7119 sha256=c9fc5e7a33cd3da689881b3112e7526437aa691a14daf5c6ae2f9797527cd1e6
  Stored in directory: /root/.cache/pip/wheels/e9/4a/1f/f9dd8e4c4b533bf6913f71ca99e3da8ec6b2d15a1a8517bf87
  Created wheel for silpa_common: filename=silpa_common-0.3-py3-none-any.whl size=8469 sha256=c0035a01e4c97f8f467641b7935df2e753ee05f6de7e13b0b913633cf1ccf650
  Stored in directory: /root/.cache/pip/wheels/1b/d2/97/c3c2

In [40]:
pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 28.3 MB/s eta 0:00:00


In [24]:
from google.colab import files

# Subir el archivo
uploaded = files.upload()

Saving df_taste_1_separado_0_1000.csv to df_taste_1_separado_0_1000 (2).csv


In [25]:
# Cargar el archivo CSV
df = pd.read_csv("df_taste_1_separado_0_1000.csv")

# Mostrar las primeras filas del DataFrame
print(df.head())

               0              1            2              3          4  \
0        Terroso          cuero   virutas...          fresa     cereza   
1         cereza   frutos rojos          NaN        Terroso     piedra   
2          fresa      frambuesa     cerez...          casis       mora   
3  No disponible            NaN          NaN  No disponible        NaN   
4        ciruela    fruta negra         c...          roble   vainilla   

                5                  6           7         8       ID  
0     arándano...  frutos del bosque     merm...       NaN  3105259  
1             NaN          chocolate      tabaco   vain...  1280319  
2   fruta negr...            Terroso   minerales    cue...  6365197  
3             NaN      No disponible         NaN       NaN  2573275  
4          tabaco              cuero     Terroso    ceniza  1133110  


In [31]:
import pandas as pd
import difflib

# Función para detectar texto incompleto
def es_incompleto(texto):
    return isinstance(texto, str) and ("..." in texto or texto.endswith("."))

# Buscar valores incompletos
valores_incompletos = set()
for col in df.columns:
    for valor in df[col]:
        if es_incompleto(valor):
            valores_incompletos.add(valor)  # Usar un conjunto para palabras incompletas

# Función para encontrar palabras similares
def encontrar_similares(df, texto, n=3):
    similares = []
    for col in df.columns:
        for valor in df[col]:
            if isinstance(valor, str):
                ratio = difflib.SequenceMatcher(None, texto, valor).ratio()
                if ratio >= 0.9:  # Umbral de similitud del 90%
                    similares.append(valor)
    return list(set(similares))[:n]  # Devolver solo n sugerencias únicas

# Contador de palabras incompletas
total_incompletas = len(valores_incompletos)
print(f"Total de palabras incompletas: {total_incompletas}")

# Pedir al usuario que complete los valores
while valores_incompletos:
    valor_incompleto = valores_incompletos.pop()  # Obtener y eliminar un valor incompleto
    print(f"\nPalabra o frase incompleta: '{valor_incompleto}'")
    sugerencias = encontrar_similares(df, valor_incompleto)

    if sugerencias:
        print("Sugerencias similares:")
        for i, sugerencia in enumerate(sugerencias, 1):
            print(f"{i}. {sugerencia}")

    nuevo_valor = input("Completa la palabra o frase (o elige una sugerencia por número): ")

    # Si el usuario elige una sugerencia
    if nuevo_valor.isdigit() and 1 <= int(nuevo_valor) <= len(sugerencias):
        nuevo_valor = sugerencias[int(nuevo_valor) - 1]
    elif nuevo_valor.strip() == "":
        print("No se ha introducido un valor. Se usará la palabra incompleta original.")
        nuevo_valor = valor_incompleto  # Mantener el valor original si no se introduce nada

    # Reemplazar en el DataFrame y mostrar cambios
    cambios_realizados = []
    for col in df.columns:
        for idx, valor in enumerate(df[col]):
            if isinstance(valor, str):
                if difflib.SequenceMatcher(None, valor, valor_incompleto).ratio() >= 0.9:  # Umbral de similitud del 90%
                    df.at[idx, col] = nuevo_valor
                    cambios_realizados.append((col, idx, valor, nuevo_valor))

    # Imprimir los cambios realizados
    if cambios_realizados:
        print("\nCambios realizados:")
        for col, idx, valor_antiguo, valor_nuevo in cambios_realizados:
            print(f"Columna: '{col}', Fila: {idx}, de '{valor_antiguo}' a '{valor_nuevo}'")

    # Actualizar el conjunto de palabras incompletas
    # Verificar si hay más palabras incompletas que coinciden con el nuevo valor
    for col in df.columns:
        for idx, valor in enumerate(df[col]):
            if isinstance(valor, str) and es_incompleto(valor):
                valores_incompletos.add(valor)  # Agregar de nuevo si sigue siendo incompleto

    print(f"Palabras incompletas restantes: {len(valores_incompletos)}")

# Mostrar el DataFrame actualizado
print("\nDataFrame actualizado:")
print(df)


Total de palabras incompletas: 273

Palabra o frase incompleta: ' hier...'
Sugerencias similares:
1.  hie...
2.  hierb...
3.  hier...
Completa la palabra o frase (o elige una sugerencia por número): hierbas

Cambios realizados:
Columna: '5', Fila: 217, de ' hie...' a 'hierbas'
Columna: '5', Fila: 370, de ' hier...' a 'hierbas'
Columna: '5', Fila: 467, de ' hier...' a 'hierbas'
Columna: '8', Fila: 14, de ' hierb...' a 'hierbas'
Columna: '8', Fila: 302, de ' hierb...' a 'hierbas'
Columna: '8', Fila: 440, de ' hierb...' a 'hierbas'
Palabras incompletas restantes: 272

Palabra o frase incompleta: ' alq...'
Sugerencias similares:
1.  alq...
Completa la palabra o frase (o elige una sugerencia por número): alquitrán

Cambios realizados:
Columna: '5', Fila: 429, de ' alq...' a 'alquitrán'
Palabras incompletas restantes: 271

Palabra o frase incompleta: ' suelo del...'
Sugerencias similares:
1.  suelo d...
2.  suelo del...
3.  suelo de...
Completa la palabra o frase (o elige una sugerencia por 

In [32]:
def es_incompleto(texto):
    return isinstance(texto, str) and ("..." in texto or texto.endswith("."))

# Contar palabras incompletas
contador_incompletas = 0

for col in df.columns:
    for valor in df[col]:
        if es_incompleto(valor):
            contador_incompletas += 1

print(f"Total de palabras incompletas o que contienen puntos suspensivos: {contador_incompletas}")


Total de palabras incompletas o que contienen puntos suspensivos: 0


In [ ]:
import pandas as pd
from spellchecker import SpellChecker
from collections import Counter

# Crear una instancia del corrector ortográfico para español
spell = SpellChecker(language='es')

# Función para verificar si una palabra está en el diccionario
def es_incompleto(texto):
    return isinstance(texto, str) and (spell.unknown([texto]) != set()) and texto.lower() != "no disponible"

# Buscar palabras incompletas
valores_incompletos = []
for col in df.columns:
    for valor in df[col]:
        if es_incompleto(valor):
            valores_incompletos.append(valor)  # Agregar a la lista de palabras incompletas

# Contar las ocurrencias de cada palabra incompleta
contador = Counter(valores_incompletos)

# Mostrar las palabras incompletas encontradas y sus conteos
print("\nPalabras incompletas encontradas y sus conteos:")
for palabra, cantidad in contador.items():
    print(f"{palabra}: {cantidad} veces")

# Solicitar al usuario las palabras a reemplazar
palabras_a_reemplazar = {}
for palabra, cantidad in contador.items():
    nueva_palabra = input(f"Ingrese la nueva palabra para '{palabra}' (aparece {cantidad} veces): ")
    palabras_a_reemplazar[palabra] = nueva_palabra

# Función para reemplazar palabras
def reemplazar_palabras(df, palabras_a_reemplazar):
    for palabra, nueva_palabra in palabras_a_reemplazar.items():
        df.replace(palabra, nueva_palabra, inplace=True)
    return df

# Reemplazar las palabras en el DataFrame
df_modificado = reemplazar_palabras(df, palabras_a_reemplazar)

# Mostrar el DataFrame modificado
print("\nDataFrame modificado:")
print(df_modificado)


Palabras incompletas encontradas y sus conteos:
aceto balsámico: 18 veces
cereza negra: 43 veces
frutos rojos: 74 veces
fruta negra: 118 veces
cereza roja: 13 veces
pimienta blanca: 7 veces
tabaco dulce: 1 veces
arándano rojo: 13 veces
grosella roja: 1 veces
suelo del bosque: 34 veces
pimiento asado: 1 veces
chocolate amargo: 8 veces
frutos del bosque: 262 veces
solución salina: 2 veces
frambuesa negra: 1 veces
ciruela pasa: 4 veces
pasas de uva negras: 2 veces
trufa negra: 5 veces
caramelo de regaliz: 5 veces
cereza Bing: 1 veces
nuez negra: 1 veces
jambul: 4 veces
virutas de lápiz: 4 veces
hierbas secas: 3 veces
clavo de olor: 48 veces
fresa salvaje: 2 veces
 cuero: 139 veces
 frutos rojos: 157 veces
 frambuesa: 134 veces
 fruta negra: 53 veces
 Terroso: 192 veces
 tabaco: 188 veces
 vainilla: 254 veces
 ciruela: 139 veces
 minerales: 48 veces
 chocolate: 95 veces
 Carne de caza: 6 veces
 arándano: 64 veces
 cedro: 53 veces
 fresa: 86 veces
 arándano rojo: 19 veces
 mora: 146 veces


In [ ]:
# Función para reemplazar palabras
def reemplazar_palabras(df, palabras_a_reemplazar):
    for palabra, nueva_palabra in palabras_a_reemplazar.items():
        df.replace(palabra, nueva_palabra, inplace=True)
    return df

# Solicitar al usuario las palabras a reemplazar
palabras_a_reemplazar = {}
while True:
    palabra = input("Ingresa la palabra a reemplazar (o 'salir' para terminar): ")
    if palabra.lower() == 'salir':
        break
    nueva_palabra = input("Ingresa la nueva palabra: ")
    palabras_a_reemplazar[palabra] = nueva_palabra

# Reemplazar las palabras en el DataFrame
df_modificado = reemplazar_palabras(df, palabras_a_reemplazar)

# Mostrar el DataFrame modificado
print("\nDataFrame modificado:")
print(df_modificado)

In [33]:
df.to_csv('datos.csv', index=False)

In [34]:
from google.colab import files

In [35]:
csv_filename = 'data.csv'
df.to_csv(csv_filename, index=False)

# Step 3: Download the CSV file
files.download(csv_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>